# Tutorial - Text Mining - Classification - NLTK

We will predict the category of discussion posts in a newsgroup.

**The unit of analysis is a discussion post**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
news = pd.read_csv('news.csv')

In [ ]:
news.head(5)

## Assign the "target" variable

This is a multi-class classification problem. There are three categories we will predict:<br>
Whether a post is "graphics," "hockey," or "medical" related

In [ ]:
target = news['newsgroup']

## Assign the "text" (input) variable

In [ ]:
# Check for missing values

news[['TEXT']].isna().sum()

In [ ]:
input_data = news['TEXT']

## Split the data

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set, train_y, test_y = train_test_split(input_data, target, test_size=0.3, random_state=42)

In [ ]:
train_set.shape, train_y.shape

In [ ]:
test_set.shape, test_y.shape

## NLTK

NLTK gives you more control over Keras Tokenizer and scikit-learn (but the idea is the same)

In [ ]:
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

## Custom tokenizer

In [ ]:
def nltk_tokenizer(dataset):

    #Create a blank list
    returned_data = []

    # For each row in dataset, we will read the text, tokenize it, remove stopwords, lemmatize it, 
    # and append it to the list

    for text in dataset:
        text = re.sub(r'[!"#$%&()*+,-./:;<=>?[\]^_`{|}~]', ' ', text).lower()
        words= nltk.tokenize.word_tokenize(text)
        words = [w for w in words if w.isalpha()]  # include only letter-based terms, exclude all terms with numbers
        words = [w for w in words if len(w)>1 and w not in stopwords.words('english')] #include terms that have more than 2 chars and not included in stopwords
        lemmatizer = nltk.stem.WordNetLemmatizer()
        words = [lemmatizer.lemmatize(w) for w in words]
        returned_data.append(' '.join(words))
    
    return returned_data

## New train/test data

In [ ]:
# Create new train and test data 

new_train = nltk_tokenizer(train_set)

new_test = nltk_tokenizer(test_set)

In [ ]:
# Convert to pandas

new_train_df = pd.DataFrame(new_train, columns=['text'])

new_test_df = pd.DataFrame(new_test, columns=['text'])

### Comparison of original and new train

In [ ]:
# original train data - first observation

train_set.iloc[0]

In [ ]:
# new train data - first observation
new_train[0]

## Use Scikit-Learn to create the term-by-doc matrix

In [ ]:
#TfidfVectorizer includes pre-processing, tokenization, filtering stop words
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(stop_words='english', max_features=500)

train_x_tfidf = tfidf_vect.fit_transform(new_train_df['text'])

In [ ]:
# Perform the TfidfVectorizer transformation
# Be careful: We are using the train fit to transform the test data set. Otherwise, the test data 
# features will be very different and match the train set!!!

test_x_tfidf = tfidf_vect.transform(new_test_df['text'])


In [ ]:
train_x_tfidf.shape, test_x_tfidf.shape

### We are not creating SVDs here. But you can if you want. It is a matter of preference. 
### Also, note that we limited the columns to 500 (by selecting the most commonly occurring terms)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

from sklearn.metrics import accuracy_score

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1) 

rnd_clf.fit(train_x_tfidf, train_y)



## Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#Train accuracy

train_y_pred = rnd_clf.predict(train_x_tfidf)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = rnd_clf.predict(test_x_tfidf)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

#Usually created on test set
confusion_matrix(test_y, test_y_pred)

## Stochastic Gradient Descent Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=100, tol=1e-3)


In [ ]:
sgd_clf.fit(train_x_tfidf, train_y)

## Accuracy

In [ ]:
#Train accuracy

train_y_pred = sgd_clf.predict(train_x_tfidf)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = sgd_clf.predict(test_x_tfidf)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

#Usually created on test set
confusion_matrix(test_y, test_y_pred)